### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data = school_data.rename(columns={'school_name': 'School Name'})
student_data = student_data.rename(columns={'school_name': 'School Name'})
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=['School Name', 'School Name'])
#school_data
#student_data
#school_data_complete

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_school = school_data['School ID'].count()

total_student = school_data['size'].sum()

total_budget = school_data['budget'].sum()

average_math = student_data['math_score'].mean()

average_reading = student_data['reading_score'].mean()

passing_math = student_data.loc[student_data['math_score'] > 70, :]

count_math = passing_math['math_score'].count()

percent_math = round(count_math/total_student * 100, 2)

passing_reading = student_data.loc[student_data['reading_score'] > 70, :]

count_reading = passing_math['reading_score'].count()

percent_reading = round(count_reading/total_student * 100, 2)

passing_both = student_data.loc[(student_data['reading_score'] > 70) & (student_data['math_score'] > 70), :]

count_both = passing_both['Student ID'].count()

percent_both = round(count_both/total_student * 100, 2)

summary_df = pd.DataFrame({'Total Schools':[total_school],
                          'Total Students':[total_student],
                          'Total Budget':[total_budget],
                          'Average Math Score':[average_math],
                          'Average Reading Score':[average_reading],
                          'Percent Passing Math':[percent_math],
                          'Percent Passing Reading':[percent_reading],
                          'Percent Passing Both':[percent_both]})
summary_df['Total Students'] = summary_df['Total Students'].map('{:,}'.format)
summary_df['Total Budget'] = summary_df['Total Budget'].map('${:,.2f}'.format)
summary_df['Average Math Score'] = summary_df['Average Math Score'].map('{:,.0f}'.format)
summary_df['Average Reading Score'] = summary_df['Average Reading Score'].map('{:,.0f}'.format)
summary_df['Percent Passing Math'] = summary_df['Percent Passing Math'].map('{:,.2f}%'.format)
summary_df['Percent Passing Reading'] = summary_df['Percent Passing Reading'].map('{:,.2f}%'.format)
summary_df['Percent Passing Both'] = summary_df['Percent Passing Both'].map('{:,.2f}%'.format)

summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
0,15,"39,170","$24,649,428.00",79,82,72.39%,72.39%,60.80%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name-
  * School Type
  * Total Students
  * Total School Budget-
  * Per Student Budget-
  * Average Math Score-
  * Average Reading Score-
  * % Passing Math-
  * % Passing Reading-
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#passing_both_summary = passing_both.rename(columns={'school_name': 'School Name'})
passing_both_summary = passing_both.groupby('School Name').count()
passing_both_summary = passing_both_summary.drop(columns=['Student ID', 'gender', 'grade', 'reading_score', 'math_score'])
passing_both_summary = passing_both_summary.rename(columns={'student_name': 'Passing Both Count'})

math_summary = passing_math.groupby('School Name').count()
math_summary = math_summary.drop(columns=['Student ID', 'gender', 'grade', 'reading_score', 'math_score'])
math_summary = math_summary.rename(columns={'student_name': 'Passing Math Count'})

reading_summary = passing_reading.groupby('School Name').count()
reading_summary = reading_summary.drop(columns=['Student ID', 'gender', 'grade', 'reading_score', 'math_score'])
reading_summary = reading_summary.rename(columns={'student_name': 'Passing Reading Count'})
reading_summary

merge_df = reading_summary.merge(math_summary, how='inner', on= 'School Name')
merge_df = merge_df.merge(passing_both_summary, how = 'inner', on= 'School Name')
merge_df = merge_df.astype(float)



In [4]:
school_data_complete = school_data_complete.rename(columns={'school_name': 'School Name',
                                                           'type': 'School Type'})
school_data = school_data.rename(columns={'school_name': 'School Name'})

school_group = school_data_complete.groupby('School Name').mean()




school_group['budget'] = school_group['budget'].astype(float)
school_group['Budget per Student'] = school_group['budget']/school_group['size']

school_summary = school_group.merge(school_data, how='inner', on= 'School Name')
school_summary = school_summary.drop(columns=['budget_y', 'Student ID', 'School ID_y', 'size_y'])
#school_group['budget'] = school_group['budget'].map('${:,.2f}'.format)
#school_group['Budget per Student'] = school_group['Budget per Student'].map('${:,.2f}'.format) 
school_summary = school_summary.rename(columns={'reading_score': 'Average Reading Score',
                                         'type': 'School Type',
                                        'math_score': 'Average Math Score',
                                        'School ID_x': 'School ID',
                                        'size_x': 'Total Students',
                                        'budget_x': 'Total School Budget',})

school_summary = school_summary.merge(merge_df, how='inner', on= 'School Name')
school_summary['Passing Both Percent'] = (school_summary['Passing Both Count']/school_summary['Total Students'])*100
school_summary['Passing Math Percent'] = (school_summary['Passing Math Count']/school_summary['Total Students'])*100
school_summary['Passing Reading Percent'] = (school_summary['Passing Reading Count']/school_summary['Total Students'])*100

school_summary_df = school_summary
school_summary = school_summary.drop(columns=['Passing Both Count', 'Passing Math Count', 'Passing Reading Count'])
school_summary = school_summary.round({'Average Reading Score': 0, 'Average Math Score': 0})

school_summary['Passing Reading Percent'] = school_summary['Passing Reading Percent'].map('{:,.2f}%'.format)
school_summary['Passing Math Percent'] = school_summary['Passing Math Percent'].map('{:,.2f}%'.format)
school_summary['Passing Both Percent'] = school_summary['Passing Both Percent'].map('{:,.2f}%'.format)
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)

school_summary

,School Name,Average Reading Score,Average Math Score,School ID,Total Students,Total School Budget,Budget per Student,School Type,Passing Both Percent,Passing Math Percent,Passing Reading Percent
0,Bailey High School,81.0,77.0,7.0,4976.0,"$3,124,928.00",628.0,District,51.15%,64.63%,79.30%
1,Cabrera High School,84.0,83.0,6.0,1858.0,"$1,081,356.00",582.0,Charter,84.02%,89.56%,93.86%
2,Figueroa High School,81.0,77.0,1.0,2949.0,"$1,884,411.00",639.0,District,49.92%,63.75%,78.43%
3,Ford High School,81.0,77.0,13.0,2739.0,"$1,763,916.00",644.0,District,51.30%,65.75%,77.51%
4,Griffin High School,84.0,83.0,4.0,1468.0,"$917,500.00",625.0,Charter,83.65%,89.71%,93.39%
5,Hernandez High School,81.0,77.0,3.0,4635.0,"$3,022,020.00",652.0,District,50.16%,64.75%,78.19%
6,Holden High School,84.0,84.0,8.0,427.0,"$248,087.00",581.0,Charter,84.07%,90.63%,92.74%
7,Huang High School,81.0,77.0,0.0,2917.0,"$1,910,635.00",655.0,District,49.91%,63.32%,78.81%
8,Johnson High School,81.0,77.0,12.0,4761.0,"$3,094,650.00",650.0,District,49.80%,63.85%,78.28%
9,Pena High School,84.0,84.0,9.0,962.0,"$585,858.00",609.0,Charter,84.82%,91.68%,92.20%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
best_schools_df = school_summary.sort_values(['Passing Both Percent'], ascending=False)
best_schools_df.head()

,School Name,Average Reading Score,Average Math Score,School ID,Total Students,Total School Budget,Budget per Student,School Type,Passing Both Percent,Passing Math Percent,Passing Reading Percent
13,Wilson High School,84.0,83.0,5.0,2283.0,"$1,319,574.00",578.0,Charter,84.89%,90.93%,93.25%
9,Pena High School,84.0,84.0,9.0,962.0,"$585,858.00",609.0,Charter,84.82%,91.68%,92.20%
14,Wright High School,84.0,84.0,10.0,1800.0,"$1,049,400.00",583.0,Charter,84.44%,90.28%,93.44%
12,Thomas High School,84.0,83.0,14.0,1635.0,"$1,043,130.00",638.0,Charter,84.28%,90.21%,92.91%
6,Holden High School,84.0,84.0,8.0,427.0,"$248,087.00",581.0,Charter,84.07%,90.63%,92.74%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
worst_schools = school_summary.sort_values(['Passing Both Percent'], ascending=True)
worst_schools.head()

,School Name,Average Reading Score,Average Math Score,School ID,Total Students,Total School Budget,Budget per Student,School Type,Passing Both Percent,Passing Math Percent,Passing Reading Percent
10,Rodriguez High School,81.0,77.0,11.0,3999.0,"$2,547,363.00",637.0,District,49.44%,64.07%,77.74%
8,Johnson High School,81.0,77.0,12.0,4761.0,"$3,094,650.00",650.0,District,49.80%,63.85%,78.28%
7,Huang High School,81.0,77.0,0.0,2917.0,"$1,910,635.00",655.0,District,49.91%,63.32%,78.81%
2,Figueroa High School,81.0,77.0,1.0,2949.0,"$1,884,411.00",639.0,District,49.92%,63.75%,78.43%
5,Hernandez High School,81.0,77.0,3.0,4635.0,"$3,022,020.00",652.0,District,50.16%,64.75%,78.19%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
ninth = school_data_complete[(school_data_complete['grade'] == '9th')]
tenth = school_data_complete[(school_data_complete['grade'] == '10th')]
eleventh = school_data_complete[(school_data_complete['grade'] == '11th')]
twelfth = school_data_complete[(school_data_complete['grade'] == '12th')]

ninth_scores = ninth.groupby(['School Name']).mean()['math_score']
tenth_scores = tenth.groupby(['School Name']).mean()['math_score']
eleventh_scores = eleventh.groupby(['School Name']).mean()['math_score']
twelfth_scores = twelfth.groupby(['School Name']).mean()['math_score']


math_grade_df = pd.DataFrame({"9th": ninth_scores, "10th":tenth_scores, "11th":eleventh_scores, "12th":twelfth_scores})
math_grade_df['9th'] = math_grade_df['9th'].map('{:,.2f}'.format)
math_grade_df['10th'] = math_grade_df['10th'].map('{:,.2f}'.format)
math_grade_df['11th'] = math_grade_df['11th'].map('{:,.2f}'.format)
math_grade_df['12th'] = math_grade_df['12th'].map('{:,.2f}'.format)

math_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
ninth_reading = ninth.groupby(['School Name']).mean()['reading_score']
tenth_reading = tenth.groupby(['School Name']).mean()['reading_score']
eleventh_reading = eleventh.groupby(['School Name']).mean()['reading_score']
twelfth_reading = twelfth.groupby(['School Name']).mean()['reading_score']


reading_grade_df = pd.DataFrame({"9th": ninth_reading, "10th":tenth_reading, "11th":eleventh_reading, "12th":twelfth_reading})
reading_grade_df['9th'] = reading_grade_df['9th'].map('{:,.2f}'.format)
reading_grade_df['10th'] = reading_grade_df['10th'].map('{:,.2f}'.format)
reading_grade_df['11th'] = reading_grade_df['11th'].map('{:,.2f}'.format)
reading_grade_df['12th'] = reading_grade_df['12th'].map('{:,.2f}'.format)



reading_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
school_summary_df.dtypes

School Name                 object
Average Reading Score      float64
Average Math Score         float64
School ID                  float64
Total Students             float64
Total School Budget        float64
Budget per Student         float64
School Type                 object
Passing Reading Count      float64
Passing Math Count         float64
Passing Both Count         float64
Passing Both Percent       float64
Passing Math Percent       float64
Passing Reading Percent    float64
dtype: object

In [10]:
school_summary_dfa = school_summary_df [['School Name',
                                        'Budget per Student',
                                        'Average Math Score',
                                        'Average Reading Score',
                                        'Passing Math Percent',
                                        'Passing Reading Percent',
                                        'Passing Both Percent']]

school_summary_dfa

,School Name,Budget per Student,Average Math Score,Average Reading Score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
0,Bailey High School,628.0,77.048432,81.033963,64.630225,79.300643,51.145498
1,Cabrera High School,582.0,83.061895,83.975780,89.558665,93.864370,84.015070
2,Figueroa High School,639.0,76.711767,81.158020,63.750424,78.433367,49.915226
3,Ford High School,644.0,77.102592,80.746258,65.753925,77.510040,51.296093
4,Griffin High School,625.0,83.351499,83.816757,89.713896,93.392371,83.651226
5,Hernandez High School,652.0,77.289752,80.934412,64.746494,78.187702,50.161812
6,Holden High School,581.0,83.803279,83.814988,90.632319,92.740047,84.074941
7,Huang High School,655.0,76.629414,81.182722,63.318478,78.813850,49.914296
8,Johnson High School,650.0,77.072464,80.966394,63.852132,78.281874,49.800462
9,Pena High School,609.0,83.839917,84.044699,91.683992,92.203742,84.823285


In [11]:



bins=[0,585,615,645,675]
bin_names = ["0-$585", "$585-615","$615-645","$645-675"]

per_student = school_summary_dfa['Budget per Student']
school_summary_dfa['School Spending per Student'] = pd.cut(per_student, bins, labels=bin_names)

reading_spending = school_summary_dfa.groupby(['School Spending per Student']).mean()['Average Reading Score']
math_spending = school_summary_dfa.groupby(['School Spending per Student']).mean()['Average Math Score']

passing_reading = school_summary_dfa.groupby(['School Spending per Student']).mean()['Passing Reading Percent']
passing_math = school_summary_dfa.groupby(['School Spending per Student']).mean()['Passing Math Percent']

passing_spending = (passing_reading + passing_math)/2

school_group['Budget per Student']
school_spenda_df = pd.DataFrame({'Average Math Score': math_spending,
                                'Average Reading Score': reading_spending,
                                'Passing Math Percent': passing_math,
                                'Passing Reading Percent': passing_reading,
                                'Passing Both Percent': passing_spending})
school_spenda_df

<ipython-input-11-983a22379cea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_summary_dfa['School Spending per Student'] = pd.cut(per_student, bins, labels=bin_names)


,Average Math Score,Average Reading Score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
School Spending per Student,,,,,
0-$585,83.455399,83.933814,90.350436,93.325838,91.838137
$585-615,83.599686,83.885211,90.788049,92.410786,91.599418
$615-645,79.079225,81.891436,73.021426,83.214343,78.117884
$645-675,76.997210,81.027843,63.972368,78.427809,71.200088


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
school_summary_dfb = school_summary_df
size_bins = [0, 2000, 3500, 5000]
size_bin_names = ["Small 0-2000", "Medium 2000-3500", "Large 3500-5000"]
total_stud = school_summary_dfb['Total Students']
school_summary_df['Scores by School Size'] = pd.cut(total_stud, size_bins, labels=size_bin_names)
size_math = school_summary_dfb.groupby(['Scores by School Size']).mean()['Average Math Score']
size_reading = school_summary_dfb.groupby(['Scores by School Size']).mean()['Average Reading Score']
size_passing_math = school_summary_dfb.groupby(['Scores by School Size']).mean()['Passing Math Percent']
size_passing_reading = school_summary_dfb.groupby(['Scores by School Size']).mean()['Passing Reading Percent']
size_passing = (size_passing_reading + size_passing_math)/2

school_size_bins= pd.DataFrame({'Average Math Score': size_math,
                                'Average Reading Score': size_reading,
                                'Passing Math Percent': size_passing_math,
                                'Passing Reading Percent': size_passing_reading,
                                'Passing Both Percent': size_passing})
school_size_bins

,Average Math Score,Average Reading Score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
Scores by School Size,,,,,
Small 0-2000,83.502373,83.883125,90.281832,93.024001,91.652916
Medium 2000-3500,78.429493,81.769122,70.938952,82.002937,76.470945
Large 3500-5000,77.063340,80.919864,64.323717,78.378664,71.351190


## Scores by School Type

* Perform the same operations as above, based on school type

In [13]:
school_summary_dfc = school_summary_df

type_math = school_summary_dfc.groupby(['School Type']).mean()['Average Math Score']
type_reading = school_summary_dfc.groupby(['School Type']).mean()['Average Reading Score']
type_passing_math = school_summary_dfc.groupby(['School Type']).mean()['Passing Math Percent']
type_passing_reading = school_summary_dfc.groupby(['School Type']).mean()['Passing Reading Percent']
type_passing = (type_passing_reading + type_passing_math)/2

school_type_bins= pd.DataFrame({'Average Math Score': type_math,
                                'Average Reading Score': type_reading,
                                'Passing Math Percent': type_passing_math,
                                'Passing Reading Percent': type_passing_reading,
                                'Passing Both Percent': type_passing})
school_type_bins

,Average Math Score,Average Reading Score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
